In [1]:
diretorio_datasets = "/home/matteus-paula/Downloads/dados_tcc/ideb"

In [2]:
import pandas as pd

> ## Dados ideb:<br> 
- Fonte: http://portal.inep.gov.br/web/guest/educacao-basica/ideb/resultados
- divulgacao_anos_iniciais-escolas-2017
- divulgacao_anos_finais-escolas-2017
- divulgacao_ensino_medio-escolas-2017

In [14]:
diretorio = f'{diretorio_datasets}/divulgacao_ensino_medio_municipios2017-atualizado-Jun_2019.xlsx'
ideb_ensino_medio = pd.read_excel(diretorio, header=None)

In [4]:
diretorio = f'{diretorio_datasets}/divulgacao_anos_iniciais_municipios2017-atualizado-Jun_2019.xlsx'
ideb_ensino_fund_iniciais = pd.read_excel(diretorio, header=None)

In [5]:
diretorio = f'{diretorio_datasets}/divulgacao_anos_finais_municipios2017-atualizado-Jun_2019.xlsx'
ideb_ensino_fund_finais = pd.read_excel(diretorio, header=None)

In [15]:
def transform_dataframe(input_df):

    header = input_df.iloc[6]
    header = header.replace(r'\n',' ', regex=True)

    dfs = input_df.iloc[10:]
    dfs.columns = header
    
    dfs_1 = dfs[['Sigla da UF', 'Código do Município',
                 'Nome do Município', 'Rede', 'IDEB 2017 (N x P)']]

    dfs_1.drop(dfs_1.tail(3).index,inplace=True)
        
    dfs_1.columns = ['SIGLA_UF', 'CO_MUNICIPIO',
                     'NO_MUNICIPIO', 'REDE', 'NOTA_IDEB']

    return dfs_1

In [16]:
out_df_ideb_ensino_medio = transform_dataframe(ideb_ensino_medio)
out_df_ideb_ensino_medio.head()

,SIGLA_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,NOTA_IDEB
10,RO,1100015,Alta Floresta D'Oeste,Estadual,3.9
11,RO,1100015,Alta Floresta D'Oeste,Pública,3.9
12,RO,1100023,Ariquemes,Pública,3.8
13,RO,1100023,Ariquemes,Estadual,3.8
14,RO,1100031,Cabixi,Estadual,3.7


In [17]:
out_df_ideb_ensino_fund_iniciais = transform_dataframe(ideb_ensino_fund_iniciais)
out_df_ideb_ensino_fund_iniciais.head()

,SIGLA_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,NOTA_IDEB
10,RO,1100015,Alta Floresta D'Oeste,Estadual,6.2
11,RO,1100015,Alta Floresta D'Oeste,Municipal,5.1
12,RO,1100015,Alta Floresta D'Oeste,Pública,6.2
13,RO,1100023,Ariquemes,Estadual,6
14,RO,1100023,Ariquemes,Municipal,5.4


In [18]:
out_df_ideb_ensino_fund_finais = transform_dataframe(ideb_ensino_fund_finais)
out_df_ideb_ensino_fund_finais.head()

,SIGLA_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,NOTA_IDEB
10,RO,1100015,Alta Floresta D'Oeste,Estadual,5.1
11,RO,1100015,Alta Floresta D'Oeste,Municipal,4.3
12,RO,1100015,Alta Floresta D'Oeste,Pública,4.8
13,RO,1100023,Ariquemes,Estadual,5
14,RO,1100023,Ariquemes,Municipal,4.7


In [23]:
out_df_ideb_ensino_medio = out_df_ideb_ensino_medio[out_df_ideb_ensino_medio['REDE'] == 'Pública']

out_df_ideb_ensino_fund_finais = out_df_ideb_ensino_fund_finais[out_df_ideb_ensino_fund_finais['REDE'] == 'Pública']

In [24]:
concat_df = [out_df_ideb_ensino_medio, out_df_ideb_ensino_fund_finais]
result_1 = pd.concat(concat_df)

result_1['NOTA_IDEB'] = pd.to_numeric(result_1['NOTA_IDEB'],errors='coerce')

columns = ['SIGLA_UF','CO_MUNICIPIO', 'NO_MUNICIPIO', 'REDE']
final_result = result_1.groupby(columns, as_index=False).mean()

In [25]:
final_result.head()

,SIGLA_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,NOTA_IDEB
0,AC,1200013,Acrelândia,Pública,4.05
1,AC,1200054,Assis Brasil,Pública,3.90
2,AC,1200104,Brasiléia,Pública,5.00
3,AC,1200138,Bujari,Pública,3.90
4,AC,1200179,Capixaba,Pública,4.00


In [26]:
final_result.isnull().any()

SIGLA_UF        False
CO_MUNICIPIO    False
NO_MUNICIPIO    False
REDE            False
NOTA_IDEB        True
dtype: bool

In [27]:
final_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5569 entries, 0 to 5568
Data columns (total 5 columns):
SIGLA_UF        5569 non-null object
CO_MUNICIPIO    5569 non-null int64
NO_MUNICIPIO    5569 non-null object
REDE            5569 non-null object
NOTA_IDEB       5536 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 261.0+ KB


In [28]:
final_result.describe()

,CO_MUNICIPIO,NOTA_IDEB
count,5.569000e+03,5536.000000
mean,3.253283e+06,3.988566
std,9.848221e+05,0.678659
min,1.100015e+06,2.100000
25%,2.512101e+06,3.500000
50%,3.146255e+06,4.000000
75%,4.119103e+06,4.500000
max,5.300108e+06,6.400000


In [29]:
dados_ideb = final_result[['CO_MUNICIPIO', 'NOTA_IDEB']][final_result['NOTA_IDEB'].notnull()]
dados_ideb.count()

CO_MUNICIPIO    5536
NOTA_IDEB       5536
dtype: int64

In [58]:
dados_ideb.to_csv(f'{diretorio_datasets}/ideb_final.csv', sep=';', encoding='utf-8', index=False)